# Install dependencies

In [ ]:
!pip install openai tiktoken
!pip install python-docx pdfplumber pymupdf
!pip install pandas openpyxl xlrd lxml
!pip install -U langchain-community
!pip install langchain_openai
!pip install chromadb
!pip install jedi
!pip install -q llama-index
!pip install -q llama-parse

# Import library

In [ ]:
# Imports necesary
import os
import json
import re
import shutil
import pickle
from io import StringIO
from typing import Dict, List
import pandas as pd
import openpyxl
from google.colab import drive
import openai
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
import nest_asyncio
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_parse import LlamaParse
from difflib import get_close_matches
from IPython.display import Image, Markdown

# Load items

## Load rutes

In [ ]:
# Mount Google Drive in Colab
drive.mount('/content/drive')

# Storage paths
output_folder = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_md_openai/"
DB_Chroma = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/Chroma_DB/"
template_path = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/CO-028296-HS-2-COSHH template.xlsx"
json_output_path = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_JSON/"

# Load utilities

In [ ]:
'''
    Loads embeddings, initializes a persistent Chroma vector store, reads the Excel template and a Markdown doc,
    and configures a GPT-4o-mini LLM—basic setup for a RAG-style retrieval + generation workflow.

    Load an embedding model compatible with the GPT-4 family
'''
API_KEY = ""
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key= API_KEY
    )

# Load the Chroma vector database
db = Chroma(persist_directory=DB_Chroma, embedding_function=embeddings)

# Load the working Excel template (all sheets)
template = pd.read_excel(template_path, sheet_name=None)

# Load the .md document (CO-028674-DS-1-3-IN-1-All-Purpose-Cleaner.md)
with open(output_folder + 'CO-028674-DS-1-3-IN-1-All-Purpose-Cleaner.md', 'r') as file:
    document = file.read()

# Load the LLM to be used
llm = ChatOpenAI (
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= API_KEY
    )

llm_gpt4o_mini = OpenAI(model="gpt-4o-mini")

# Extract fields from Excel as JSON

In [ ]:
# Path to the Markdown file to be processed`
ruta_table_2_Hazards = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_2.md"
ruta_table_3_Waste_disposal_measures = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_3.md"
ruta_table_4_Spill_management = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_4.md"
ruta_table_5_Fire_procedures = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_5.md"
ruta_table_6_First_aid_procedures = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_6.md"
ruta_table_7_Storage = "/content/drive/MyDrive/TFM/Grupo_1_RAG_Chemical_Safety/Notebooks/output_Excel_to_md/table_7.md"

In [ ]:
# Read UTF-8 text files for all section tables (Hazards, Waste disposal, Spill management, Fire procedures,
# First aid, Storage) from their file paths into memory as strings.

with open(ruta_table_2_Hazards, "r", encoding="utf-8") as f:
    table_Hazards = f.read()

with open(ruta_table_3_Waste_disposal_measures, "r", encoding="utf-8") as f:
    table_Waste_disposal_measures = f.read()

with open(ruta_table_4_Spill_management, "r", encoding="utf-8") as f:
    table_Spill_management = f.read()

with open(ruta_table_5_Fire_procedures, "r", encoding="utf-8") as f:
    table_Fire_procedures = f.read()

with open(ruta_table_6_First_aid_procedures, "r", encoding="utf-8") as f:
    table_First_aid_procedures = f.read()

with open(ruta_table_7_Storage, "r", encoding="utf-8") as f:
    table_Storage = f.read()

# Convert Markdown to JSON

In [ ]:
'''
Instructs the AI to read the safety sheet (Markdown) and list every field the Excel should include,
returning a plain JSON template with empty values, including the “Who might be harmed?” items.
'''

prompt_text_sheet2 = """
You have the following Markdown content, which comes from a converted Excel sheet. Your task is to identify all the fields that should be filled in the Excel.
The Excel is intended to be auto-completed later and represents a Safety Data Sheet.

Return a JSON whose keys correspond to those fields, all with empty values. Do NOT invent additional information.
Keep in mind that within the same sheet there may be several table fragments, so review every cell that contains information.

Make sure to go through all relevant cells and follow this EXACT structure:

{{{{
  "Sheet_2": {{{{
    "campo1": {{"content": "string", "position": "string", "response": "string", "to_excel":"string"}},
    "campo2": {{"content": "string", "position": "string", "response": "string", "to_excel":"string"}},
    "campo3": {{"content": "string", "position": "string", "response": "string", "to_excel":"string"}},
  }}}}
}}}}

For fields like the one containing "Who might be harmed?", there are 3 fixed cells with information: Operators, Bystanders, Maintenance Personnel. These fields must follow the structure below:
"who_might_be_harmed_operators": {
      "content": "Operators",
      "position": "",
      "response": "",
      "to_excel": ""
    },
    "who_might_be_harmed_bystanders": {
      "content": "Bystanders",
      "position": "",
      "response": "",
      "to_excel": ""
    },
    "who_might_be_harmed_maintenance_personnel": {
      "content": "Maintenance Personnel",
      "position": "",
      "response": "",
      "to_excel": ""
    }

Where:
- Keys must be derived only from keywords, with no punctuation, no numbers, and words separated by underscores.
- `content` must contain the full text of the source cell, such as titles, questions, or headers.
- `response` must always be empty.
- `position` must always be empty.
- `to_excel` must always be empty.

Return only the raw JSON, with no additional formatting such as json or explanations.

--- MARKDOWN ---
{markdown}
--- FIN ---
"""

In [ ]:
def convert_excel_to_json(prompt_text, markdown_text):
    '''
    Runs an LLM prompt on the given Markdown to extract a JSON template for the Excel sheet,
    prints the raw LLM response for debugging, parses it into clean JSON, and returns the result,
    raises an error if the response isn’t valid JSON.
    '''

    # Create the LangChain prompt using the received text
    prompt_extract_keys = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You specialize in extracting structured data from Markdown to Excel spreadsheets.",
            ),
            ("human", prompt_text),
        ]
    )

    chain_extract_keys = prompt_extract_keys | llm

    # Execute markdown query
    response = chain_extract_keys.invoke({"markdown": markdown_text})

    # Display content in case of errors or for debugging
    print(response.content)

    # Convert to clean JSON
    try:
        json_sheet = json.loads(response.content)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error al parsear JSON: {e}\nContenido:\n{response.content}")

    # Return the final dictionary
    return json_sheet

In [ ]:
# Generate the JSON template for the Hazards section from the Markdown table via the LLM
json_table_Hazards = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_Hazards
)
print(json.dumps(json_table_Hazards, indent=2, ensure_ascii=False))

In [ ]:
# Generate the JSON template for the Waste disposal measures section from the Markdown table via the LLM
json_table_Waste_disposal_measures = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_Waste_disposal_measures
)
print(json.dumps(json_table_Waste_disposal_measures, indent=2, ensure_ascii=False))

In [ ]:
# Generate the JSON template for the Spill management section from the Markdown table via the LLM
json_table_Spill_management = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_Spill_management
)
print(json.dumps(json_table_Spill_management, indent=2, ensure_ascii=False))

In [ ]:
# Generate the JSON template for the Fire procedures section from the Markdown table via the LLM
json_table_Fire_procedures = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_Fire_procedures
)
print(json.dumps(json_table_Fire_procedures, indent=2, ensure_ascii=False))

In [ ]:
# Generate the JSON template for the Aid procedures section from the Markdown table via the LLM
json_table_First_aid_procedures = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_First_aid_procedures
)
print(json.dumps(json_table_First_aid_procedures, indent=2, ensure_ascii=False))

In [ ]:
# Generate the JSON template for the Storage section from the Markdown table via the LLM
json_table_Storage = convert_excel_to_json(
    prompt_text=prompt_text_sheet2,
    markdown_text=table_Storage
)
print(json.dumps(json_table_Storage, indent=2, ensure_ascii=False))

# Map to excel

In [ ]:
# define constants for each field’s cell reference

map_table_Hazards = {
    "hazards": "",
    "chemical_name": "C13",
    "hazard_group": "D13",
    "sds_reference": "E13",
    "physical_form_and_quantity":"F13",
    "hazard_statements":"G13",
    "explosive":"H13",
    "flammable": "I13",
    "oxidising": "J13",
    "gas_under_pressure": "K13",
    "acute_toxicity": "L13",
    "corrosive": "M13",
    "health_hazard": "N13",
    "serious_health_hazard":"O13",
    "hazardous_to_the_environment":"P13",
    "potential_routes_of_exposure":"Q13",
    "workplace_exposure_limits":"R13",
    "arising_harm": "S13",
    "who_might_be_harmed_operators": "",
    "who_might_be_harmed_bystanders": "",
    "who_might_be_harmed_maintenance_personnel": "",
    "severity": ["U13","U14","U15"],
    "likelihood_before_control_measures":["V13","V14","V15"],
    "risk_before_control_measures": "", # Campos autocalculados, no mapeados de momento
    "wear_full_face_visor": "X13",
    "box_goggles_must_be_worn": "Y13",
    "protective_gloves_must_be_worn": "Z13",
    "laboratory_coats_must_be_worn": "AA13",
    "use_local_exhaust_ventillation": "AB13",
    "no_open_flames": "AC13",
    "other_control_measures": "AD13",
    "actions_control_measures_to_be_implemented": "AE13",
    "assign_to": "AF13",
    "completion_date": "AG13",
    "likelihood_after_control_measures": ["AH13","AH14","AH15"],
    "risk_after_control_measures": "" # Campos autocalculados, no mapeados de momento
}

In [ ]:
map_table_Waste_disposal_measures = {
    "waste_disposal_measures": "",
    "chemical_name": "C24",
    "hazard_group": "D24",
    "handling_of_the_product_if_it_becomes_waste": "F24",
    "do_not_dispose_of_via_drain": ""
  }

In [ ]:
map_table_Spill_management = {
   "spill_management": "",
    "chemical_name": "Q24",
    "details": "R24"
}

In [ ]:
map_table_Fire_procedures = {
    "fire_procedures": "",
    "chemical_name": "Q30",
    "details": "R30"
}

In [ ]:
map_table_First_aid_procedures = {
    "first_aid_procedures": "",
    "chemical_name": "C38",
    "eyes": "D38",
    "skin": "H38",
    "if_ingested": "L38",
    "if_inhaled": "Q38"
}

In [ ]:
map_table_Storage = {
    "chemical_name": "C30",
    "hazard_group": "D30",
    "hazard_label_and_store_safely_on_shelf": "E30",
    "flammables_cupboard": "F30",
    "corrosives_cupboard": "G30",
    "poisons_cupboard": "H30",
    "ventilated_storage": "I30",
    "gas_cylinder": "J30",
    "cold_storage": "K30",
    "dessicated_storage": "L30",
    "special_storage_describe": "M30"
}

In [ ]:
def set_positions_in_json(json_data, mapping, json_output_path, output_filename, sheet_name="Sheet_2"):
    '''
        Fills each field’s "position" in the JSON from the given mapping (list or string),
        warns if a key is missing, pretty-prints the final JSON, saves it to disk,
        and returns the updated JSON object.
    '''
    for key, position in mapping.items():
        if key in json_data.get(sheet_name, {}):
            if isinstance(position, list):
                json_data[sheet_name][key]["position"] = position
            else:
                json_data[sheet_name][key]["position"] = str(position)
        else:
            print(f"⚠️ La clave '{key}' no se encontró en el JSON.")

    # Display the final JSON only once
    print(json.dumps(json_data, indent=2, ensure_ascii=False))

    # Save the JSON to the specified path
    os.makedirs(json_output_path, exist_ok=True)
    output_file = os.path.join(json_output_path, output_filename)
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)

    print(f"✅ JSON guardado en: {output_file}")
    return json_data

In [ ]:
set_positions_in_json(
    json_table_Hazards,
    map_table_Hazards,
    json_output_path,
    output_filename="json_table_Hazards.json"
)

In [ ]:
set_positions_in_json(
    json_table_Waste_disposal_measures,
    map_table_Waste_disposal_measures,
    json_output_path,
    output_filename="json_table_Waste_disposal_measures.json"
)

In [ ]:
set_positions_in_json(
    json_table_Spill_management,
    map_table_Spill_management,
    json_output_path,
    output_filename="json_table_Spill_management.json"
)

In [ ]:
set_positions_in_json(
    json_table_Fire_procedures,
    map_table_Fire_procedures,
    json_output_path,
    output_filename="json_table_Fire_procedures.json"
)

In [ ]:
set_positions_in_json(
    json_table_First_aid_procedures,
    map_table_First_aid_procedures,
    json_output_path,
    output_filename="json_table_First_aid_procedures.json"
)

In [ ]:
set_positions_in_json(
    json_table_Storage,
    map_table_Storage,
    json_output_path,
    output_filename="json_table_Storage.json"
)